Poyecto 1: Didier Irias Mendez

In [620]:
# importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor


In [621]:
# Loading the CSV in a Dataframe
df = pd.read_csv("../datasets/Uncleaned_DS_jobs.csv", encoding ="latin-1")

Creating the working table

In [622]:
# Checking if there are null values on the data
df.isnull().sum()
# Removing duplicated data in case that exists
if df.duplicated().sum():
    df.drop_duplicates(inplace=True)


In [623]:
# Cleaning Salary Estimate column
df['Salary Estimate'] = df['Salary Estimate'].str.split(' ').str[0]

In [624]:
# Deleting the unnecessary column for the regression
df = df.drop(columns=['index','Rating', 'Size', 'Founded', 'Industry', 'Competitors', 'Headquarters', 'Type of ownership', 'Revenue', 'Company Name', 'Job Description'])


In [625]:
# deleting -1 rows
columns_with_default_values = df.columns[df.apply(lambda x: x.astype(str).str.contains('-1').any())]
df = df.loc[~((df[columns_with_default_values] == -1) | (df[columns_with_default_values] == "-1")).any(axis=1)]
df = df.dropna(subset=columns_with_default_values)

Standarizing Job Title

In [626]:
"""
The Job Titles are categorizied into 6 columns in order to standardized the data for the regression.
- Technical Manager
- Data Scientist
- Data Engineer
- Machine Learning
- Data Analyst
- Other
"""
df.loc[df['Job Title'].str.contains('Manager', case=False), 'Job Title'] = 'Technical Manager'
df.loc[df['Job Title'].str.contains('Data Scientist', case=False), 'Job Title'] = 'Data Scientist'
df.loc[df['Job Title'].str.contains('Data Engineer', case=False), 'Job Title'] = 'Data Engineer'
df.loc[df['Job Title'].str.contains('Machine Learning', case=False), 'Job Title'] = 'Machine Learning'
df.loc[df['Job Title'].str.contains('Data Analyst', case=False), 'Job Title'] = 'Data Analyst'
df.loc[~df['Job Title'].str.contains('Data Scientist|Manager|Data Engineer|Machine Learning|Data Analyst', case=False), 'Job Title'] = 'Other'

df

,Job Title,Salary Estimate,Location,Sector
0,Data Scientist,$137K-$171K,"New York, NY",Insurance
1,Data Scientist,$137K-$171K,"Chantilly, VA",Business Services
2,Data Scientist,$137K-$171K,"Boston, MA",Business Services
3,Data Scientist,$137K-$171K,"Newton, MA",Manufacturing
4,Data Scientist,$137K-$171K,"New York, NY",Business Services
...,...,...,...,...
665,Data Scientist,$105K-$167K,"Vienna, VA",Information Technology
666,Data Scientist,$105K-$167K,"Boston, MA",Biotech & Pharmaceuticals
667,Data Scientist,$105K-$167K,"Fort Lee, NJ",Business Services
670,Data Scientist,$105K-$167K,"San Francisco, CA",Business Services


In [627]:
# Creating dependent variables
"""
To do the regression, 3 dependent variables were created
- Salary_Lower: The minimum salary for that Job Title
- Salary_Upper: The maximum salary for that Job Title
- Average_Salary: The Avarage salary for that Job Title
"""
df[['Salary_Lower', 'Salary_Upper']] = df['Salary Estimate'].str.extract(r'\$(\d+\.?\d*)K-\$(\d+\.?\d*)K')
df['Salary_Lower'] = df['Salary_Lower'].astype(float) * 1000
df['Salary_Upper'] = df['Salary_Upper'].astype(float) * 1000
df['Average_Salary'] = (df['Salary_Lower'] + df['Salary_Upper']) / 2
average_salary = df['Average_Salary'].mean()
df

,Job Title,Salary Estimate,Location,Sector,Salary_Lower,Salary_Upper,Average_Salary
0,Data Scientist,$137K-$171K,"New York, NY",Insurance,137000.0,171000.0,154000.0
1,Data Scientist,$137K-$171K,"Chantilly, VA",Business Services,137000.0,171000.0,154000.0
2,Data Scientist,$137K-$171K,"Boston, MA",Business Services,137000.0,171000.0,154000.0
3,Data Scientist,$137K-$171K,"Newton, MA",Manufacturing,137000.0,171000.0,154000.0
4,Data Scientist,$137K-$171K,"New York, NY",Business Services,137000.0,171000.0,154000.0
...,...,...,...,...,...,...,...
665,Data Scientist,$105K-$167K,"Vienna, VA",Information Technology,105000.0,167000.0,136000.0
666,Data Scientist,$105K-$167K,"Boston, MA",Biotech & Pharmaceuticals,105000.0,167000.0,136000.0
667,Data Scientist,$105K-$167K,"Fort Lee, NJ",Business Services,105000.0,167000.0,136000.0
670,Data Scientist,$105K-$167K,"San Francisco, CA",Business Services,105000.0,167000.0,136000.0


In [628]:
# Saving the new cleaned data set ready for the regression
df.to_csv('../datasets/cleaned_DS-jobs.csv', index=False)

Making the Multiple Linear Regression

In [629]:
# loading the Cleaned Data
df = pd.read_csv('../datasets/cleaned_DS-jobs.csv')
df

,Job Title,Salary Estimate,Location,Sector,Salary_Lower,Salary_Upper,Average_Salary
0,Data Scientist,$137K-$171K,"New York, NY",Insurance,137000.0,171000.0,154000.0
1,Data Scientist,$137K-$171K,"Chantilly, VA",Business Services,137000.0,171000.0,154000.0
2,Data Scientist,$137K-$171K,"Boston, MA",Business Services,137000.0,171000.0,154000.0
3,Data Scientist,$137K-$171K,"Newton, MA",Manufacturing,137000.0,171000.0,154000.0
4,Data Scientist,$137K-$171K,"New York, NY",Business Services,137000.0,171000.0,154000.0
...,...,...,...,...,...,...,...
596,Data Scientist,$105K-$167K,"Vienna, VA",Information Technology,105000.0,167000.0,136000.0
597,Data Scientist,$105K-$167K,"Boston, MA",Biotech & Pharmaceuticals,105000.0,167000.0,136000.0
598,Data Scientist,$105K-$167K,"Fort Lee, NJ",Business Services,105000.0,167000.0,136000.0
599,Data Scientist,$105K-$167K,"San Francisco, CA",Business Services,105000.0,167000.0,136000.0


In [630]:
"""
NOTE: I also selected Sector and Location as independent variables because I think these varibles might affect the salary, 
since depending of the sector or location the salary might increase or decrease.
"""
# Select the independent variables and the dependent variable
X = df[['Job Title', 'Salary Estimate', 'Sector', 'Location']]
y = df[['Salary_Lower', 'Salary_Upper', 'Average_Salary']]

In [631]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Location', 'Sector', 'Salary_Lower',
       'Salary_Upper', 'Average_Salary'],
      dtype='object')

In [632]:
# Perform one-hot encoding on the independent variables columns  to transform categorical variables into float
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X).toarray()

# Split the data into training and testing sets, 80% of data for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Create and fit the multiple linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on the testing data
y_pred = model.predict(X_test)  


In [633]:
# R-squared
r2 = r2_score(y_test, y_pred)

# Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
# Root Mean Squared Error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

print("R-squared:", r2)
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)

R-squared: 0.9967532659103989
MSE: 4967093.430658591
RMSE: 2202.3906280595033
MAE: 829.6609230623368


Decision Tree

In [634]:

"""
NOTE: I also selected Sector and Location as independent variables because I think these varibles might affect the salary, 
since depending of the sector or location the salary might increase or decrease.
"""
# Select the independent variables and the dependent variable
X = df[['Job Title', 'Salary Estimate', 'Sector', 'Location']]
y = df[['Salary_Lower', 'Salary_Upper', 'Average_Salary']]

# Perform one-hot encoding on the independent variables columns
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X).toarray()

# Split the data into training and testing sets, 80% of data for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Create and fit the decision tree regressor model
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)

# Predict on the testing data
y_pred = model.predict(X_test)


Randon Forest

In [635]:
"""
NOTE: Since I believe the Decision Tree is overfitting I decided to create a random forest, which results seems to be better.
"""
from sklearn.ensemble import RandomForestRegressor

# Select the independent variables and the dependent variable
X = df[['Job Title', 'Salary Estimate', 'Sector', 'Location']]
y = df[['Salary_Lower', 'Salary_Upper', 'Average_Salary']]

# Perform one-hot encoding on the independent variables columns
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X).toarray()

# Split the data into training and testing sets, 80% of data for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Create and fit the Random Forest regressor model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predict on the testing data
y_pred = model.predict(X_test)



Decision Tree - Acurracy

In [636]:
# Calculate R-squared
r2 = r2_score(y_test, y_pred)
# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
# Calculate RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Print the metrics
print("R-squared:", r2)
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)

R-squared: 0.9999859584062681
MSE: 21138.84297520661
RMSE: 144.3137078762504
MAE: 34.710743801652896
